In [1]:
import re
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import xfeat
from xfeat.cat_encoder import TargetEncoder
import sklearn
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import optuna.integration.lightgbm as lgb

In [2]:
all_new = pd.read_csv('data/feature_df_agg_svd64.csv')
#bert_embeded = pd.read_csv('data/bert_embeded.csv')
bert_svd256 = pd.read_csv("data/bert_svd256.csv")
ALBERT_svd256 = pd.read_csv("data/ALBERT_svd256.csv")
ROBERTA_svd256 = pd.read_csv("data/ROBERTA_svd256.csv")
XLMROBERTA_svd256 = pd.read_csv("data/XLMROBERTA_svd256.csv")
#tfidf_svd_raw_64 = pd.read_csv("data/tfidf_svd_raw_64.csv")
tfidf_svd_raw_64 = pd.read_csv("data/tfidf_svd_raw_128_2gram.csv")
mfw_df = pd.read_csv('data/mfw.csv')
miw_df = pd.read_csv('data/miw.csv')

In [3]:
#all_new = pd.merge(all_new, bert_embeded, on="id", how="outer")
#all_new = pd.merge(all_new, bert_svd256, on="id", how="outer")
#all_new = pd.merge(all_new, ROBERTA_svd256, on="id", how="outer")
all_new = pd.merge(all_new, tfidf_svd_raw_64, on="id", how="outer")
all_new = pd.merge(all_new, miw_df, on="id", how="outer")
all_new = pd.merge(all_new, mfw_df.drop(["mfw", "sfw"], axis=1), on="id", how="outer")

In [4]:
all_new.head(3)

,id,country,duration,category1,category2,state,data_type,number_of_chars,number_of_words,number_of_sentences,...,wordvec_15_y,wordvec_16_y,wordvec_17_y,wordvec_18_y,wordvec_19_y,wordvec_20_y,wordvec_21_y,wordvec_22_y,wordvec_23_y,wordvec_24_y
0,0,4,29,12,143,0.0,train,5289,961,40,...,0.10287,-0.17618,-1.28810,-0.598010,0.26131,-1.26190,0.392020,0.59309,-0.55232,0.005087
1,1,16,34,5,108,0.0,train,1144,202,9,...,0.35207,0.16624,-0.91364,0.693300,-0.65035,0.77353,-0.087782,-0.43228,-0.99982,-0.680770
2,2,21,30,7,122,0.0,train,3316,549,25,...,0.35500,0.16465,0.32006,-0.045544,-0.31911,-1.01380,-0.054359,0.20074,0.32061,-0.455840


In [5]:
def run_lgbm_with_te(test, train, target_cols, feature_cols, categorical_cols):
    X_train = train[feature_cols]
    y_train = train[target_cols]
    X_test = test[feature_cols]
    test_idx = X_test.index
    train_idx = X_train.index
    
    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))
    importances = []
    
    cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=123)
    kfold = KFold(n_splits=10, shuffle=True)
    cfold_test = ConstantFold([range(len(train_idx))], [range(len(train_idx)+1, len(train_idx)+len(test_idx))])
    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, X_train["category2"])):
        X_tr = X_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        X_tr = pd.concat([X_tr, y_tr], axis=1)
        X_tr = X_tr.reset_index(drop=True)

        X_tr = get_te_features(X_tr,
                                   categorical_cols=categorical_cols,
                                   target_col=target_cols,
                                   fold=kfold)
        X_tr = X_tr.drop(target_cols, axis=1)
        
        X_val = X_train.iloc[valid_index]
        y_val = y_train.iloc[valid_index]
        X_val = pd.concat([pd.concat([X_train.iloc[train_index], y_tr], axis=1), X_val],
                            axis=0)
        X_val = X_val.reset_index(drop=True)

        cfold_val = ConstantFold([range(len(train_index))], [range(len(train_index)+1, len(train_index)+len(valid_index))])
        X_val = get_te_features(X_val,
                                categorical_cols=categorical_cols,
                               target_col=target_cols,
                               fold=cfold_val)
        
        X_val = X_val.drop(target_cols, axis=1)
        X_val = X_val.iloc[-len(valid_index):]  # concatしてるのでX_val.iloc[valid_index]としてはダメ
        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)
        
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=False,
                          num_boost_round=1000,
                          early_stopping_rounds=50,
                          )
        
        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        
        X_test_te = pd.concat([X_train, y_train], axis=1)
        X_test_te = pd.concat([X_test_te, X_test], axis=0)
        X_test_te = X_test_te.reset_index(drop=True)
        X_test_te = get_te_features(X_test_te,
                                   categorical_cols=categorical_cols,
                                   target_col=target_cols,
                                   fold=cfold_test)
        X_test_te = X_test_te.drop(target_cols, axis=1)
        X_test_te = X_test_te.iloc[-len(test_idx):]
        y_pred = model.predict(X_test_te,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)
        
        
        # display importance
        importance = pd.DataFrame(model.feature_importance(), index=X_test_te.columns, columns=['importance'])
        display(importance)
        importances.append(importance)

    return oof_train, sum(y_preds) / len(y_preds), importances

In [6]:
def run_optuna_lgbm(test, train, target_cols, feature_cols, categorical_cols):
    """
    import optuna.integration.lightgbm as lgb
    """
    X_train = train[feature_cols]
    y_train = train[target_cols]
    X_test = test[feature_cols]
    
    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))
    importances = []
    
    #cv = KFold(n_splits=5, shuffle=True, random_state=0)
    cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=123)
    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, X_train["category2"])):
        X_tr = X_train.iloc[train_index]
        X_val = X_train.iloc[valid_index]
        y_tr = y_train.iloc[train_index]
        y_val = y_train.iloc[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)
        
        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=False,
                          num_boost_round=1000,
                          early_stopping_rounds=10,
                          )
        
        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)
        
        
        # display importance
        importance = pd.DataFrame(model.feature_importance(), index=X_test.columns, columns=['importance'])
        display(importance)
        importances.append(importance)

    return oof_train, sum(y_preds) / len(y_preds), importances

In [7]:
def get_categorical_cols(base_categorical_cols, feature_cols):
    selected_cols = list(base_categorical_cols)
    for col in base_categorical_cols:
        if col not in feature_cols:
            selected_cols.remove(col)
            print(f"remove: {col}")
        else:
            print(f"chosen: {col}")
    return selected_cols


def get_te_features(df_origin, categorical_cols, target_col, fold):
    df = df_origin.copy()
    encoder = TargetEncoder(
        input_cols=categorical_cols, 
        target_col=target_col,
        fold=fold,
        output_prefix="te_",
        output_suffix=""
        )
    output_df = encoder.fit_transform(df)
    return output_df


# target endordingをtest setに適用するためのクラス
class ConstantFold(KFold):
    def __init__(self, train_idx, val_idx):
        self.train_idx = np.array(train_idx)
        self.val_idx = np.array(val_idx)
        if len(self.train_idx) != len(self.val_idx):
            raise RuntimeError(f"length of 'train_idx'({len(self.train_idx)}) and 'val_idx'({len(self.val_idx)}) must be same.")
        
    def split(self, X):
        whole_idx = pd.DataFrame(X).index
        for tr_idx, val_idx in zip(self.train_idx, self.val_idx):
            tr_idx, val_idx = whole_idx[tr_idx], whole_idx[val_idx]
            yield tr_idx, val_idx

In [8]:
FEATURE_COLS = list(all_new.columns.drop(["id", "state","data_type"]))
print(len(FEATURE_COLS))
CATEGORICAL_COLS = [
    "country",
    "category1",
    "category2",
    "country+category1",
    "country+category2",
    "category1+category2",
]


907


In [9]:
TARGET_COLS = ["state"]
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
    }

In [10]:
test_set = all_new[all_new["data_type"] == "test"]
train_set = all_new[all_new["data_type"] == "train"]
#_oof, _preds, importances = run_optuna_lgbm(test_set, train_set, target_cols, feature_cols, categorical_cols)

In [11]:
def feature_selection(test_set, train_set, target_cols, feature_cols, categorical_cols, num_features):
    _, _, importances = run_optuna_lgbm(test_set, train_set, target_cols, feature_cols, categorical_cols)
    features_set = set()
    for importance in importances:
        imp_df = importance.sort_values("importance")
        feature_selected = list(imp_df[-num_features:].index)
        features_set |= set(feature_selected)
    return list(features_set), importances

In [12]:
def recursive_feature_selection(test_set, train_set, target_cols, feature_cols, categorical_cols):
    worst_exist = True
    while worst_exist == True:
        oof, preds, importances = run_optuna_lgbm(test_set, train_set, target_cols, feature_cols, categorical_cols)
        #oof, preds, importances = run_lgbm_with_te(test_set, train_set, target_cols, feature_cols, categorical_cols)
        worst_features = check_worst_features(importances, 5)
        if worst_features == set():
            worst_exist = False
        else:
            for feature in worst_features:
                feature_cols.remove(feature)
                for category in categorical_cols:
                    if category not in feature_cols:
                        print(category)
                        categorical_cols.remove(category)
    return oof, preds, importances, feature_cols

In [13]:
def check_worst_features(importances, imp_threshold):
    worst_feature_set = set(importances[0].index)
    for n, importance in enumerate(importances):
        df = importance.sort_values('importance')
        worst_features = set(df[df["importance"] < imp_threshold].index)
        worst_feature_set &= worst_features
    return worst_feature_set

In [14]:
"""
feature_selected, importances = feature_selection(test_set, train_set, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS, 150)
categorical_selected = get_categorical_cols(CATEGORICAL_COLS, feature_selected)
feature_selected
"""

'\nfeature_selected, importances = feature_selection(test_set, train_set, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS, 150)\ncategorical_selected = get_categorical_cols(CATEGORICAL_COLS, feature_selected)\nfeature_selected\n'

In [ ]:
#oof, preds, importances = run_lgbm_with_te(test_set, train_set, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS)
#oof, preds, importances = run_optuna_lgbm(test_data, train_data, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS)
_oof, _preds, _importances, feature_cols = recursive_feature_selection(test_set, train_set, TARGET_COLS, FEATURE_COLS, CATEGORICAL_COLS)
feature_cols

/home/teppei/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=7.
  warnings.warn(("The least populated class in y has only %d"
[I 2021-01-29 10:23:21,835] A new study created in memory with name: no-name-2e178414-2319-4746-aa67-f2ef2ca08d6c
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 4562, number of negative: 4476
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050368 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 157152
[LightGBM] [Info] Number of data points in the train set: 9038, number of used features: 906


/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/teppei/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504758 -> initscore=0.019031
[LightGBM] [Info] Start training from score 0.019031


feature_fraction, val_score: 0.407327:  14%|#4        | 1/7 [00:02<00:13,  2.21s/it]

...

[LightGBM] [Info] Number of positive: 4542, number of negative: 4497
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009784 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66736
[LightGBM] [Info] Number of data points in the train set: 9039, number of used features: 281
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502489 -> initscore=0.009957
[LightGBM] [Info] Start training from score 0.009957


feature_fraction, val_score: 0.406800:  43%|####2     | 3/7 [00:01<00:02,  1.99it/s]

[LightGBM] [Info] Number of positive: 4542, number of negative: 4497
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66736
[LightGBM] [Info] Number of data points in the train set: 9039, number of used features: 281
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502489 -> initscore=0.009957
[LightGBM] [Info] Start training from score 0.009957


feature_fraction, val_score: 0.406800:  57%|#####7    | 4/7 [00:02<00:01,  2.05it/s]

[LightGBM] [Info] Number of positive: 4542, number of negative: 4497
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007505 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 66736
[LightGBM] [Info] Number of data points in the train set: 9039, number of used features: 281
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502489 -> initscore=0.009957
[LightGBM] [Info] Start training from score 0.009957


In [ ]:
def optimize_accuracy(oof, y_train):
    best_score = 0
    scores = []
    ratios = []
    for thr in np.arange(0, 1.0, 0.025):
        oof_ = np.where(oof<thr, 0, 1)
        ratio = np.count_nonzero(oof_>0)/len(oof_)
        ratios.append(ratio)
        accuracyscore = accuracy_score(y_true=y_train, y_pred=oof_)
        scores.append(accuracyscore)
        best_score = max(best_score, accuracyscore)
        if accuracyscore == best_score:
            best_threshold = thr
    plt.figure()
    plt.title("accuracy_score")
    plt.scatter(np.arange(0, 1.0, 0.025), scores)
    plt.scatter(np.arange(0, 1.0, 0.025), ratios)
    return best_score, best_threshold

In [ ]:
best_score, best_threshold = optimize_accuracy(_oof, list(train_set["state"]))
print(best_score, best_threshold)

In [ ]:
def optimize_threshold(oof, y_train):
    best_score = 0
    scores = []
    ratios = []
    for thr in np.arange(0, 1.0, 0.025):
        oof_ = np.where(oof<thr, 0, 1)
        ratio = np.count_nonzero(oof_>0)/len(oof_)
        ratios.append(ratio)
        f1score = f1_score(y_true=y_train, y_pred=oof_,average='binary', sample_weight=None, zero_division='warn')
        scores.append(f1score)
        best_score = max(best_score, f1score)
        if f1score == best_score:
            best_threshold = thr
    plt.figure()
    plt.title('f1_score')
    plt.scatter(np.arange(0, 1.0, 0.025), scores)
    plt.scatter(np.arange(0, 1.0, 0.025), ratios)
    return best_score, best_threshold

In [ ]:
best_score, best_threshold = optimize_threshold(_oof, list(train_set["state"]))
print(best_score, best_threshold)
#0.8122529644268774 0.4 orginal
#0.8102929099370381 0.425 without agg

In [ ]:
cm = confusion_matrix(list(train_set["state"]), np.where(_oof<0.4, 0, 1))
cmd = ConfusionMatrixDisplay(cm)
cmd.plot()

In [ ]:
def making_sub(preds, threshold):
    preds = np.where(preds<threshold, 0, 1)
    
    df = pd.DataFrame(columns=["id", "state"])
    df["state"] = preds
    df["id"] = range(10545, 21089)
    return df

def making_preds(preds1, preds2):
    preds1 += preds2
    preds1 /= 2
    #df = pd.DataFrame(columns=["id", "state"])
    #df["state"] = preds1
    #df["id"] = range(10545, 21089)
    return preds1

In [ ]:
sub_file = making_sub(_preds, best_threshold)
sub_file.to_csv(f"./sub_files/optuna_lgbm_sub_{48}.csv", index=False, header=False)
plt.figure()
sub_file["state"].value_counts().plot(kind="bar")

In [ ]:
config_dict = {
    "selected_feature": FEATURE_COLS,
    "out_of_fold": list(_oof),
    "prediction": list(_preds),
}

In [ ]:
with open("data/lgbm_config_svd128_raw_2gram_agg_svd64.json", 'w') as f:
    json.dump(config_dict, f)